In [ ]:
import json
import sys
from dotenv import load_dotenv, find_dotenv

sys.path.append('../../aisuite')

# Load from .env file if available
load_dotenv(find_dotenv())

In [ ]:
# Mock tool functions.
def get_current_temperature(location: str, unit: str):
    # Simulate fetching temperature from an API
    return {"location": location, "unit": unit, "temperature": 72}

In [ ]:
def is_it_raining(location: str):
    """Check if it is raining at a location.

    Args:
        location (str): Name of the Place.

    Returns:
        bool: Whether it is raining in that place.
    """
    return True

In [ ]:
model = "anthropic:claude-3-5-sonnet-20240620"
# model = "openai:gpt-4o"
# model = mistral:mistral-large-latest
# model = "aws:anthropic.claude-3-haiku-20240307-v1:0"
# model = "aws:meta.llama3-1-8b-instruct-v1:0"
# model = "aws:meta.llama3-3-70b-instruct-v1:0"
# model = "groq:llama-3.1-70b-versatile"

In [ ]:
from aisuite import Client, ToolManager  # Import your ToolManager class

client = Client()
tool_manager = ToolManager([get_current_temperature, is_it_raining])

messages = [{"role": "user", "content": "What is the current temperature in San Francisco in Celsius, and is it raining?"}]

response = client.chat.completions.create(
    model=model, messages=messages, tools=tool_manager.tools()) # tool_choice="auto", parallel_tool_calls=True)

In [ ]:
from pprint import pprint
pprint(response.choices[0].message)

In [ ]:
if response.choices[0].message.tool_calls:
    tool_results, result_as_message = tool_manager.execute_tool(response.choices[0].message.tool_calls)
    messages.append(response.choices[0].message) # Model's function call message
    messages.extend(result_as_message)

    response = client.chat.completions.create(
        model=model, messages=messages, tools=tool_manager.tools())
    print(response.choices[0].message.content)
    pprint(response.choices[0].message)
else:
    pprint(response.choices[0].message)

In [ ]:
if response.choices[0].message.tool_calls:
    tool_results, result_as_message = tool_manager.execute_tool(response.choices[0].message.tool_calls)
    messages.append(response.choices[0].message) # Model's function call message
    messages.extend(result_as_message)

    response = client.chat.completions.create(
        model=model, messages=messages, tools=tool_manager.tools())
    print(response.choices[0].message.content)
else:
    pprint(response.choices[0].message)

In [ ]:
# Now, test without tool calling, to check that the normal path is not broken.
messages = [{"role": "user", "content": "What is the capital of California?"}]
response = client.chat.completions.create(
    model=model, messages=messages)
print(response.choices[0].message.content)

In [ ]:
pprint(tool_manager.tools())

In [ ]:
from aisuite import Client, ToolManager
ToolManager([get_current_temperature, is_it_raining]).tools()